<a href="https://colab.research.google.com/github/TheDataFestAI/Learning_Resources/blob/main/learning_poc/poc1_ml_clv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POC for CLV (Customer Life Cycle Value)

## Reference:

1. Domain Knowledge:
    1. https://www.analyticsvidhya.com/blog/2020/10/a-definitive-guide-for-predicting-customer-lifetime-value-clv/
    2. https://towardsdatascience.com/data-driven-growth-with-python-part-3-customer-lifetime-value-prediction-6017802f2e0f
    3. https://www.kaggle.com/code/shailaja4247/customer-lifetime-value-prediction

## Step 1: Get the source data from Kaggle

Ref:

### Install the required python packages

In [1]:
! pip install -q kaggle

In [2]:
# from google.colab import userdata
# userdata.get('kaggle_auth')
